In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

In [2]:
RGGI_states = ['CT', 'DE', 'ME', 'MD', 'MA', 'NH', 'NJ', 'NY', 'RI', 'VT'] #, 'VA','PA'

In [3]:
## Reading in new data

In [4]:
from bs4 import BeautifulSoup 
import requests

In [5]:
eia_html = requests.get('https://www.eia.gov/electricity/data/eia860m/')

In [98]:
soup = BeautifulSoup(eia_html.content,'html.parser')

In [7]:
release_dates= soup.find_all('div',class_='release-dates')

In [8]:
release_dates[0].find_all('span',class_='label')[-1]

<span class="label">Next Release Date:</span>

In [9]:
next_update_time = pd.to_datetime(release_dates[0].find_all('span',class_='date')[-1].text,format='%B %d, %Y')

In [11]:
X = soup.find_all('a', href=True)

In [12]:
monthly_report_links = {}
for x in soup.find_all('a', href=True):
    try:
        if '860M' in x.attrs['title']:
            report_date= pd.to_datetime(x.attrs['title'][9:],format='%B %Y')
            monthly_report_links[report_date] = x.attrs['href']
    except:
        pass


In [13]:
link_df = pd.DataFrame.from_dict(monthly_report_links,orient='index').sort_index()

In [101]:
link_df.index[-1]

Timestamp('2024-09-01 00:00:00')

In [14]:
recent_report = pd.ExcelFile('https://www.eia.gov'+link_df.iloc[-1].values[0],engine='openpyxl')

Operating
Canceled or Postponed
Operating_PR
Planned
Planned_PR
Retired
Retired_PR


In [15]:
plants = pd.read_excel(recent_report,sheet_name='Operating',header=2)

In [16]:
plants = plants.dropna(subset=['Entity Name'])

In [17]:
RGGI_plants =  plants.query(f'`Plant State`=={RGGI_states} and `Nameplate Capacity (MW)`>25 ')

In [18]:
Fossil_tech =  ['Petroleum Liquids', 'Natural Gas Steam Turbine','Natural Gas Fired Combined Cycle',
                'Conventional Steam Coal','Natural Gas Fired Combustion Turbine','Other Gases']   

In [19]:
RGGI_plants['Fossil'] =  np.where(RGGI_plants['Technology'].isin(Fossil_tech),1,0)

C:\Users\Alex Child\AppData\Local\Temp\ipykernel_22340\1768610179.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  RGGI_plants['Fossil'] =  np.where(RGGI_plants['Technology'].isin(Fossil_tech),1,0)


In [20]:
RGGI_plants[['year','month']] = RGGI_plants[['Planned Retirement Year','Planned Retirement Month']]

C:\Users\Alex Child\AppData\Local\Temp\ipykernel_22340\3734419072.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  RGGI_plants[['year','month']] = RGGI_plants[['Planned Retirement Year','Planned Retirement Month']]
C:\Users\Alex Child\AppData\Local\Temp\ipykernel_22340\3734419072.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  RGGI_plants[['year','month']] = RGGI_plants[['Planned Retirement Year','Planned Retirement Month']]


In [21]:
RGGI_plants['year'] = RGGI_plants['year'].astype(str).str.rstrip().replace('',2100).astype(int)


C:\Users\Alex Child\AppData\Local\Temp\ipykernel_22340\1458454439.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  RGGI_plants['year'] = RGGI_plants['year'].astype(str).str.rstrip().replace('',2100).astype(int)


In [22]:
RGGI_plants['month'] = RGGI_plants['month'].astype(str).str.rstrip().replace('',12).astype(int)

C:\Users\Alex Child\AppData\Local\Temp\ipykernel_22340\249150701.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  RGGI_plants['month'] = RGGI_plants['month'].astype(str).str.rstrip().replace('',12).astype(int)


In [23]:
RGGI_plants['day']=1

C:\Users\Alex Child\AppData\Local\Temp\ipykernel_22340\2424427767.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  RGGI_plants['day']=1


In [24]:
RGGI_plants['retirement_date'] = pd.to_datetime(RGGI_plants[['year','month','day']])

C:\Users\Alex Child\AppData\Local\Temp\ipykernel_22340\1308019628.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  RGGI_plants['retirement_date'] = pd.to_datetime(RGGI_plants[['year','month','day']])


In [25]:
RGGI_plants.columns

Index(['Entity ID', 'Entity Name', 'Plant ID', 'Plant Name', 'Google Map',
       'Bing Map', 'Plant State', 'County', 'Balancing Authority Code',
       'Sector', 'Generator ID', 'Unit Code', 'Nameplate Capacity (MW)',
       'Net Summer Capacity (MW)', 'Net Winter Capacity (MW)', 'Technology',
       'Energy Source Code', 'Prime Mover Code', 'Operating Month',
       'Operating Year', 'Planned Retirement Month', 'Planned Retirement Year',
       'Status', 'Nameplate Energy Capacity (MWh)', 'DC Net Capacity (MW)',
       'Planned Derate Year', 'Planned Derate Month',
       'Planned Derate of Summer Capacity (MW)', 'Planned Uprate Year',
       'Planned Uprate Month', 'Planned Uprate of Summer Capacity (MW)',
       'Latitude', 'Longitude', 'Fossil', 'year', 'month', 'day',
       'retirement_date'],
      dtype='object')

In [26]:
px.bar(RGGI_plants.query('Fossil==1 and retirement_date<"2100-01-01"'),x='retirement_date',color='Technology',y='Nameplate Capacity (MW)',barmode='group',hover_data=['Plant State','Plant Name'],
       title='RGGI Planned Fossil Retirements up until 2100')

In [27]:
RGGI_plants.groupby(['Balancing Authority Code','Technology'])['Nameplate Capacity (MW)'].sum().sort_values()

Balancing Authority Code  Technology                          
NBSO                      Wood/Wood Waste Biomass                    26.8
ISNE                      Offshore Wind Turbine                      30.0
NBSO                      Onshore Wind Turbine                       42.0
NYIS                      Wood/Wood Waste Biomass                    84.5
PJM                       Other Gases                               130.0
NYIS                      Offshore Wind Turbine                     130.0
                          Solar Photovoltaic                        171.5
PJM                       Onshore Wind Turbine                      180.0
NYIS                      Municipal Solid Waste                     205.8
PJM                       Municipal Solid Waste                     242.8
                          Solar Photovoltaic                        259.1
ISNE                      Solar Photovoltaic                        363.4
                          Municipal Solid Waste  

## Planned

In [28]:
planned = pd.read_excel(recent_report,sheet_name='Planned',header=2)

In [29]:
RGGI_planned = planned.query(f'`Plant State`=={RGGI_states}')

In [30]:
RGGI_planned[['year','month']] = RGGI_planned[['Planned Operation Year','Planned Operation Month']]
RGGI_planned['year'] = RGGI_planned['year'].astype(str).str.rstrip().replace('',2100).astype(float)
RGGI_planned['month'] = RGGI_planned['month'].astype(str).str.rstrip().replace('',12).astype(float)
RGGI_planned['day']=1
RGGI_planned['operational_date'] = pd.to_datetime(RGGI_planned[['year','month','day']])

C:\Users\Alex Child\AppData\Local\Temp\ipykernel_22340\1352928395.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Alex Child\AppData\Local\Temp\ipykernel_22340\1352928395.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Alex Child\AppData\Local\Temp\ipykernel_22340\1352928395.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.p

In [31]:
approved_status_list = ['(U) Under construction, less than or equal to 50 percent complete',
       '(TS) Construction complete, but not yet in commercial operation',
       '(V) Under construction, more than 50 percent complete',
       '(T) Regulatory approvals received. Not under construction',
       ]

In [32]:
not_yet_approved_status_list = ['(P) Planned for installation, but regulatory approvals not initiated',
       '(L) Regulatory approvals pending. Not under construction',
       '(OT) Other']

In [33]:
full_status_list = ['(U) Under construction, less than or equal to 50 percent complete',
       '(TS) Construction complete, but not yet in commercial operation',
       '(V) Under construction, more than 50 percent complete',
       '(T) Regulatory approvals received. Not under construction',
       '(P) Planned for installation, but regulatory approvals not initiated',
       '(L) Regulatory approvals pending. Not under construction',
       '(OT) Other']

In [34]:
additions = RGGI_planned.query(f"Status=={full_status_list}").pivot_table(index='operational_date',columns='Technology',values='Nameplate Capacity (MW)',aggfunc=np.sum).fillna(0)

In [35]:
approved_additions = RGGI_planned.query(f"Status=={approved_status_list}").pivot_table(index='operational_date',columns='Technology',values='Nameplate Capacity (MW)',aggfunc=np.sum).fillna(0)
not_yet_approved_additions = RGGI_planned.query(f"Status=={not_yet_approved_status_list}").pivot_table(index='operational_date',columns='Technology',values='Nameplate Capacity (MW)',aggfunc=np.sum).fillna(0)

In [36]:
px.area(additions.cumsum(),labels={'value':'Capacity (MW)'},title='RGGI Planned Capacity At all stages',width=900)

In [37]:
px.area(approved_additions.cumsum(),labels={'value':'Capacity (MW)'},title='Approved RGGI State Capacity Pipeline',width=900)

In [38]:
px.area(not_yet_approved_additions.cumsum(),labels={'value':'Capacity (MW)'},title='Not Yet Approved RGGI State Capacity Pipeline',width=900)

In [39]:
px.area(approved_additions.cumsum(),labels={'value':'Capacity (MW)'},title='Approved RGGI State Capacity Pipeline',width=900)

In [40]:
rggi_capacity_by_tech = RGGI_plants.groupby('Technology')['Nameplate Capacity (MW)'].sum()

In [41]:
px.bar(rggi_capacity_by_tech)

In [42]:
from datetime import datetime as dt

In [43]:
today_string = dt.today().strftime('%Y-%m-%d').split('-')

In [44]:
time_span = pd.date_range(start=dt.today().strftime('%Y-%m-%d'),end='2030-12-01',freq='MS') #

In [45]:
full_capacity_time_series = pd.DataFrame(index=time_span,columns= plants.Technology.unique())

In [46]:
rggi_capacity_by_tech_today = rggi_capacity_by_tech.to_frame().transpose()

In [47]:
rggi_capacity_by_tech_today.index.name='Date'

In [48]:
rggi_capacity_by_tech_today.index = [f'{today_string[0]}-{today_string[1]}-01']

In [49]:
rggi_capacity_by_tech_today.index  = pd.to_datetime(rggi_capacity_by_tech_today.index)

In [50]:
rggi_capacity_by_tech_today

Technology,Conventional Hydroelectric,Conventional Steam Coal,Hydroelectric Pumped Storage,Municipal Solid Waste,Natural Gas Fired Combined Cycle,Natural Gas Fired Combustion Turbine,Natural Gas Steam Turbine,Nuclear,Offshore Wind Turbine,Onshore Wind Turbine,Other Gases,Petroleum Liquids,Solar Photovoltaic,Wood/Wood Waste Biomass
2024-11-01,4414.7,2374.9,3287.0,831.1,38798.3,8597.3,12433.9,12381.4,160.0,4113.5,130.0,10541.6,794.0,737.6


In [51]:
full_capacity_time_series = pd.concat([full_capacity_time_series,rggi_capacity_by_tech_today],axis=0).sort_index()

In [52]:
full_capacity_time_series = full_capacity_time_series.fillna(0)

In [53]:
px.bar(full_capacity_time_series)

In [54]:
## Combining

In [55]:
RGGI_retirements = RGGI_plants.query('Fossil==1 and retirement_date<"2100-01-01"').pivot_table(index='retirement_date',columns='Technology',values='Nameplate Capacity (MW)',aggfunc=np.sum)

In [56]:
rggi_plus_retirements = pd.concat([full_capacity_time_series,-RGGI_retirements],axis=0).sort_index().fillna(0)

In [57]:
rggi_plus_retirements_plus_additions = rggi_plus_retirements.add(additions.loc[rggi_plus_retirements.index[0]:rggi_plus_retirements.index[-1]],fill_value=0)

In [123]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import plotly.subplots as sp

In [ ]:
px.bar(RGGI_plants.query('Fossil==1 and retirement_date<"2100-01-01"'),x='retirement_date',color='Technology',y='Nameplate Capacity (MW)',barmode='group',hover_data=['Plant State','Plant Name'],
       title='RGGI Planned Fossil Retirements up until 2100')

In [165]:
this_figure.data

(Bar({
     'alignmentgroup': 'True',
     'customdata': array([['DE', 'Indian River Generating Station'],
                          ['MD', 'Brandon Shores'],
                          ['MD', 'Brandon Shores']], dtype=object),
     'hovertemplate': ('Technology=Conventional Steam ' ... '{customdata[1]}<extra></extra>'),
     'legendgroup': 'Conventional Steam Coal',
     'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
     'name': 'Conventional Steam Coal',
     'offsetgroup': 'Conventional Steam Coal',
     'orientation': 'v',
     'showlegend': False,
     'textposition': 'auto',
     'x': array([datetime.datetime(2026, 12, 31, 0, 0),
                 datetime.datetime(2028, 12, 31, 0, 0),
                 datetime.datetime(2028, 12, 31, 0, 0)], dtype=object),
     'xaxis': 'x',
     'y': array([445.5, 685.1, 685.1]),
     'yaxis': 'y'
 }),
 Bar({
     'alignmentgroup': 'True',
     'customdata': array([['MD', 'Herbert A Wagner'],
                          ['MD', 'Herbert A

In [168]:
this_figure.data[0]

Bar({
    'alignmentgroup': 'True',
    'customdata': array([['DE', 'Indian River Generating Station'],
                         ['MD', 'Brandon Shores'],
                         ['MD', 'Brandon Shores']], dtype=object),
    'hovertemplate': ('Technology=Conventional Steam ' ... '{customdata[1]}<extra></extra>'),
    'legendgroup': 'Conventional Steam Coal',
    'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
    'name': 'Conventional Steam Coal',
    'offsetgroup': 'Conventional Steam Coal',
    'orientation': 'v',
    'showlegend': False,
    'textposition': 'auto',
    'x': array([datetime.datetime(2026, 12, 31, 0, 0),
                datetime.datetime(2028, 12, 31, 0, 0),
                datetime.datetime(2028, 12, 31, 0, 0)], dtype=object),
    'xaxis': 'x',
    'y': array([445.5, 685.1, 685.1]),
    'yaxis': 'y'
})